In [ ]:
import math
import ast
import numpy as np
from shapely.geometry import Polygon,Point
from shapely.validation import make_valid
from matplotlib import pyplot as plt
import pandas as pd
from tqdm import tqdm
import json
import geopandas as gpd
np.seterr(divide='ignore', invalid='ignore')
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
import utm #pip install utm
from pyproj import CRS
from shapely.geometry import Polygon, Point, LineString
from math import atan2, degrees
import time
from shapely import wkt
import re

In [ ]:
# Function to convert Footprint2D string to list of points
def footprint_to_polygon(footprint_str):
    points = [tuple(list(map(float, point.split(',')))[::-1]) for point in footprint_str.split()]
    return Polygon(points)
def convert_footprint_to_points(footprint):
    points = footprint.split('_')
    point_tuples = [tuple(map(float, point.split('/'))) for point in points]
    return point_tuples
def convert_polygon_to_tuples(polygon):
    # Extract the exterior coordinates of the polygon
    coords = list(polygon.exterior.coords)
    return coords
def calculate_geometry_metrics(coords, geom_type='Polygon'):
    if geom_type == 'Polygon':
        geom = Polygon(coords)
        centroid = geom.centroid
        area = geom.area
        return area, (centroid.x, centroid.y)
    elif geom_type == 'LineString':
        geom = LineString(coords)
        length = geom.length
        midpoint_x = (coords[0][0] + coords[1][0]) / 2
        midpoint_y = (coords[0][1] + coords[1][1]) / 2
        return length, (midpoint_x, midpoint_y)
def calculate_azimuth(point1, point2):
    dx = point2[0] - point1[0] # x is longtitude, this calculates diff in long
    dy = point2[1] - point1[1] # latitude
    azimuth = degrees(atan2(dx, dy))
    azimuth = (azimuth+90)% 360  # Ensure the angle is between 0 and 360 degrees
    return azimuth

def calculate_angle_with_x_axis(p1, p2, p3):
    v1 = np.array(p2) - np.array(p1)
    v2 = np.array(p3) - np.array(p1)
    normal_vector = np.cross(v1, v2)
    x_axis = np.array([1, 0, 0])
    dot_product = np.dot(normal_vector, x_axis)
    normal_vector_magnitude = np.linalg.norm(normal_vector)
    cos_angle = dot_product / (normal_vector_magnitude * np.linalg.norm(x_axis))
    cos_angle = np.clip(cos_angle, -1.0, 1.0)
    angle = np.arccos(cos_angle) * (180 / np.pi)
    return angle

def calculate_normal_vector(points):
    v1 = points[1] - points[0]
    v2 = points[2] - points[0]
    return np.cross(v1, v2)


def rotate_to_xy(points):
    normal_vector = calculate_normal_vector(points)
    theta = -np.arctan2(normal_vector[1], normal_vector[0])
    if np.isclose(theta, np.pi/2, atol=1e-2) or np.isclose(theta, -np.pi/2, atol=1e-2):
        rotated_points = points[:, [0, 2]]
    elif np.isclose(theta, 0, atol=1e-2) or np.isclose(theta, np.pi, atol=1e-2) or np.isclose(theta, -np.pi, atol=1e-2):
        rotated_points = points[:, [1, 2]]
    else:
        Rz = np.array([
            [np.cos(theta), -np.sin(theta), 0],
            [np.sin(theta), np.cos(theta), 0],
            [0, 0, 1]
        ])
        rotated_points = np.dot(points, Rz.T)
        rotated_points = rotated_points[:, 1:]
    return rotated_points, theta

def rotate_follow(points, theta):
    if np.isclose(theta, np.pi/2, atol=1e-2) or np.isclose(theta, -np.pi/2, atol=1e-2):
        rotated_points = points[:, [0, 2]]
    elif np.isclose(theta, 0, atol=1e-2) or np.isclose(theta, np.pi, atol=1e-2) or np.isclose(theta, -np.pi, atol=1e-2):
        rotated_points = points[:, [1, 2]]
    else:
        theta_rad = np.radians(theta) if abs(theta) > np.pi else theta
        Rz = np.array([
            [np.cos(theta_rad), -np.sin(theta_rad), 0],
            [np.sin(theta_rad), np.cos(theta_rad), 0],
            [0, 0, 1]
        ])
        rotated_points = np.dot(points, Rz.T)
        rotated_points = rotated_points[:, 1:]
    return rotated_points


def get_cos_theta(points):
    normal_vector = calculate_normal_vector(points)
    theta = -np.arctan2(normal_vector[1], normal_vector[0])
    cos_theta = np.cos(theta)
    # 定义阈值来判断 cos_theta 是否接近0
    threshold = 1e-2
    if np.abs(cos_theta) < threshold:
        return 1
    else:
        return cos_theta
        
def calculate_intersections(PA, PB, D):
    nA = calculate_normal_vector(PA)
    nB = calculate_normal_vector(PB)
    dA = -np.dot(nA, PA[0])
    dB = -np.dot(nB, PB[0])

    if abs(np.dot(nA, PA[3]) + dA) > 1e-6:
        raise ValueError('Fourth point does not lie on Plane A')
    if abs(np.dot(nB, PB[3]) + dB) > 1e-6:
        raise ValueError('Fourth point does not lie on Plane B')

    intersections = np.zeros((4, 3))
    for i in range(4):
        t = -(np.dot(nB, PA[i]) + dB) / np.dot(nB, D)
        intersections[i] = PA[i] + t * D
    return intersections

def is_behind(source, target, sun_direction):
    target_avg = (np.array(target[0]) + np.array(target[1])) / 2
    source_avg = (np.array(source[0]) + np.array(source[1])) / 2
    vector_c = target_avg - source_avg
    return np.dot(vector_c, sun_direction) < 0

def calculate_cosine_2d(points, sun):
    p1, p2, p3 = np.array(points[0]), np.array(points[1]), np.array(points[2])
    v31 = p3 - p1
    v21 = p2 - p1
    normal = np.cross(v31, v21)
    normal_2d = normal[:2] 
    sun_2d = sun[:2]
    l_normal_2d = np.linalg.norm(normal_2d)
    l_sun_2d = np.linalg.norm(sun_2d)
    dian_2d = np.dot(normal_2d, sun_2d)
    cos_2d = dian_2d / (l_normal_2d * l_sun_2d)
    return cos_2d

def convert_to_list(item):
    if isinstance(item, np.ndarray):
        return item.tolist()
    elif isinstance(item, dict):
        return {key: convert_to_list(value) for key, value in item.items()}
    elif isinstance(item, list):
        return [convert_to_list(element) for element in item]
    else:
        return item

In [ ]:
# read solar position
As_list_pd = pd.read_excel('data/solar azimuth angle.xlsx')
As_list = As_list_pd['Solar\nAzimuth:'].to_list()
hs_list = As_list_pd['Solar\nElevation:'].to_list()
solar_x, solar_y, solar_z = As_list_pd['solar_pos_x'].astype(float).to_list(), As_list_pd['solar_pos_y'].astype(float).to_list(), As_list_pd['solar_pos_z'].astype(float).to_list()

# read building footprint
raw = pd.read_csv('data/SF_building.csv',index_col=None)
raw['geometry'] = raw['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(raw, geometry='geometry')
gdf.set_crs(epsg=32610, inplace=True)
gdf['coordinates'] = gdf['coordinates'].apply(ast.literal_eval)
gdf.head(2)

## main code

In [ ]:
# building partition

xxx_xy=[]
xxx=[]
xxx_gaodu=[]
for index, row in gdf.iterrows():
    border_points = row['coordinates']
    elevation = row['elevation_m']
    height = row['Height'] + row['elevation_m']
    
    x, y = zip(*border_points)
    x=list(x)
    y=list(y)
    arrayx = np.array(x)[::-1]
    arrayy = np.array(y)[::-1]
    
    # z=np.zeros(len(list(x)))    
    elevation = [elevation]*(len(list(x)))
    z_height=[height]*(len(list(x)))
    
    zuobiao_xy=zip(list(arrayx),list(arrayy))
    zuobiao=zip(list(arrayx),list(arrayy),list(elevation))
    zuobiao_gaodu=zip(list(arrayx),list(arrayy),list(z_height))
    xxx_xy.append(list(zuobiao_xy))
    xxx.append(list(zuobiao))
    xxx_gaodu.append(list(zuobiao_gaodu))    

num_cell = 1
buildings_analysis_xy=[ [ [] for j in range(num_cell) ] for i in range(num_cell) ]
buildings_analysis=[ [ [] for j in range(num_cell) ] for i in range(num_cell) ]
buildings_analysis_height=[ [ [] for j in range(num_cell) ] for i in range(num_cell) ]
bounds = [[0]*num_cell for _ in range(num_cell)]

offset = 100
lat_max = test['lat_utm'].max()+offset
lat_min = test['lat_utm'].min()-offset
lon_max = test['lon_utm'].max()+offset
lon_min = test['lon_utm'].min()-offset
width = lon_max - lon_min  # width in meters
height = lat_max - lat_min  # height in meters
area = width * height
lat_step = (lat_max - lat_min) / num_cell
lon_step = (lon_max - lon_min) / num_cell
num_building = 0
for i in range(0,num_cell,1):
    for j in range(0,num_cell,1):
        bounds[i][j] = [lon_min + i * lon_step, lat_min + j * lat_step, lon_min + (i + 1) * lon_step, lat_min + (j + 1) * lat_step]
        for t in range(len(xxx)):
            #filter the buildings
            center=np.mean(np.array(xxx[t]),axis=0)
            if center[0]> bounds[i][j][0] and center[0] <=  bounds[i][j][2] and center[1] >  bounds[i][j][1] and center[1] <=  bounds[i][j][3]:
                buildings_analysis_xy[i][j].append(xxx_xy[t])
                buildings_analysis[i][j].append(xxx[t])
                buildings_analysis_height[i][j].append(xxx_gaodu[t])
                num_building +=1
for i in range(num_cell):
    for j in range(num_cell):
        if buildings_analysis[i][j]:  
            print(f"Grid cell ({i}, {j}) contains {len(buildings_analysis[i][j])} buildings")

In [ ]:
test_case

In [ ]:
num_coords = test['coordinates'].apply(len)
num_coords.describe()

In [ ]:
def parse_coordinates(coord_string):
    # Extract all number pairs from the string
    pairs = re.findall(r'\(([-+]?\d*\.\d+|\d+),\s*([-+]?\d*\.\d+|\d+)\)', coord_string)
    # Convert strings to floats
    return [tuple(map(float, pair)) for pair in pairs]

# Apply the function to your column
parsed_coords = test['coordinates'].apply(parse_coordinates)

# Get the number of coordinate pairs in each row
num_coords = parsed_coords.apply(len)

In [ ]:
parts_count = 
parts_count

### this only calculates roof

In [ ]:
buildings_analyzed = 0
unique_building_id = 0
results = {}
building_loc_metadata = {}
building_loc = []
start_time = time.time()
for r in range(0,num_cell,1):
    for w in range(0,num_cell,1):
        for q in tqdm(range(unique_building_id, 2)):
        # for q in tqdm(range(unique_building_id, len(buildings_analysis[r][w]))):
            building_target = np.array(buildings_analysis[r][w][q])
            building_target_xy = np.array(buildings_analysis_xy[r][w][q])
            building_target_dingmian = np.array(buildings_analysis_height[r][w][q])
            s_roof_value = 0
            original_roof_area, roof_centroid = calculate_geometry_metrics(building_target_xy, geom_type='Polygon')
            original_facade_areas = []
            facade_azimuths = []
            facade_loc = []
            building_loc.append(building_target_xy)
            
            cell_key = f'cell_{r}_{w}'
            building_key = f'{unique_building_id}_building_{round(original_roof_area)}'
            cell_key_loc = f'cell_{r}_{w}'
            building_key_loc = f'building_{unique_building_id}'
            if cell_key not in building_loc_metadata:
                building_loc_metadata[cell_key] = {}
            if building_key not in building_loc_metadata[cell_key]:
                building_loc_metadata[cell_key][building_key] = {
                    'roof_loc': None,
                    'facade_loc': {},
                    'facade_azimuth': {},          
                }
            for u in range(len(building_target) - 1):
                segment_length, segment_midpoint = calculate_geometry_metrics([building_target[u], building_target[u + 1]], geom_type='LineString')
                height = building_target_dingmian[u][2]  # Assuming the height is the z-coordinate
                facade_area = segment_length * height
                original_facade_areas.append(facade_area)
                azimuth = calculate_azimuth(building_target[u], building_target[u + 1])
                facade_azimuths.append(azimuth)
                facade_loc.append(segment_midpoint)
                building_loc_metadata[cell_key][building_key]['facade_azimuth'][f'{unique_building_id}_{round(original_roof_area)}_{u}_{round(original_facade_areas[u])}'] = azimuth

            # log building metadata (location, azimuth)
            building_loc_metadata[cell_key][building_key]['roof_loc'] = building_target_xy
            building_loc_metadata[cell_key][building_key]['facade_loc'] = facade_loc
            
            # Calculate cos values for BASE coordinates
            cos_=[]
            for l in range(0,len(building_target)-1,1):
                f_l=np.array(building_target_xy[l+1])-np.array(building_target_xy[l])
                xx=np.array([1,0])
                l_xx=np.sqrt(xx.dot(xx))
                l_f_l=np.sqrt(f_l.dot(f_l))
                dian=xx.dot(f_l)
                cos_.append(dian/(l_xx*l_f_l))

            p1=Polygon(building_target_xy)     
            xmin=p1.bounds[0]
            ymin=p1.bounds[1]
            xmax=p1.bounds[2]
            ymax=p1.bounds[3]
            cx1=(xmin+xmax)/2
            cy1=(ymin+ymax)/2
            pc1=Point(cx1,cy1)
            
            round_r = max(pc1.distance(Point(coord)) for coord in building_target_xy)
            
            lat=38
            p=math.pi

            for k in range (164,165):
                for i in range(0,24):
                    hs = hs_list[i] 
                    if hs<=0:
                        continue
                    As = As_list[i]
                    # sun=np.array([-math.sin(As*p/180)*math.cos(hs*p/180),-math.cos(As*p/180)*math.cos(hs*p/180),-math.sin(hs*p/180)])
                    sun = np.array([-solar_x[i], -solar_y[i], -solar_z[i]])
                    s_roof=Polygon()
                    s_facade=[Polygon()]*(len(building_target)-1)
                    s_facade_value=[0]*(len(building_target)-1)
                    u_box=[]
                    us_box=[]
                    for u in range(0,len(building_target)-1,1):
                        points = [building_target[u], building_target[u+1], building_target_dingmian[u]]
                        cosine_value = calculate_cosine_2d(points, sun)
                        if cosine_value > 0: 
                            u_box.append(u)
                        elif cosine_value  < 0:
                            us_box.append(u)               
                    u_mian=u_box

                    for h in range(0,len(u_mian),1):    
                        mm=u_mian[h]
                        Planefrom4points = [building_target[mm],building_target[mm+1],building_target_dingmian[mm+1],building_target_dingmian[mm]]
                        rotated_Planefrom4points, theta = rotate_to_xy(np.array(Planefrom4points))
                        polygon_facade = Polygon(rotated_Planefrom4points)
                        #for e in range(0,len(us_box),1):
                        #    u=us_box[e]
                        for u in range(0,len(building_target)-1,1):
                            if u!=mm: 
                                Planefrom4points_us = [building_target[u],building_target[u+1],building_target_dingmian[u+1],building_target_dingmian[u]]
                                if is_behind(Planefrom4points_us, Planefrom4points, sun):
                                    continue        
                                intersections = calculate_intersections(np.array(Planefrom4points_us), np.array(Planefrom4points), sun)
                                rotated_Coordinate4points = rotate_follow(intersections, theta)
                                targetbd = Polygon(rotated_Coordinate4points)
                                new_s_facade = s_facade[mm].union(targetbd)
                                s_facade[mm] = new_s_facade
                        s_facade[mm] = polygon_facade.intersection(s_facade[mm]) 
                        s_facade_value[mm] = s_facade[mm].area
    
                    for n in range(len(buildings_analysis[r][w])):
                        if n==q:
                            continue
                        if n!=q:
                            building_near_xy=buildings_analysis_xy[r][w][n]
                            building_near=buildings_analysis[r][w][n]
                            building_near_dingmian=buildings_analysis_height[r][w][n]
                        cxn=(Polygon(building_near_xy).bounds[0]+Polygon(building_near_xy).bounds[2])/2
                        cyn=(Polygon(building_near_xy).bounds[1]+Polygon(building_near_xy).bounds[3])/2
                        pcn=Point(cxn,cyn)
                        dis_bds=pc1.distance(pcn)-round_r
                        Hmax=building_near_dingmian[0][2]/math.tan(hs*p/180)

                        if dis_bds > Hmax: #or Polygon(building_near_xy).bounds[2] <= p1.bounds[0] or Polygon(building_near_xy).bounds[3] <= p1.bounds[1]:
                            continue
                            
                        z_box=[]    
                        for z in range(0,len(building_near)-1,1):
                            points = [building_near[z], building_near[z+1], building_near_dingmian[z]]
                            cosine_value = calculate_cosine_2d(points, sun)
                            if cosine_value < 0: 
                                z_box.append(z)  
                        
                        for e in range(0,len(u_box),1):
                            u=u_box[e]
                            Planefrom4points = [building_target[u],building_target[u+1],building_target_dingmian[u+1],building_target_dingmian[u]]
                            rotated_Planefrom4points, theta = rotate_to_xy(np.array(Planefrom4points))
                            polygon_facade = Polygon(rotated_Planefrom4points)
                            for g in range(0,len(z_box),1):
                                vertex = z_box[g]
                                Planefrom4points_us = [building_near[vertex],building_near[vertex+1],building_near_dingmian[vertex+1],building_near_dingmian[vertex]]      
                                if is_behind(Planefrom4points_us, Planefrom4points, sun):
                                    continue        
                                intersections = calculate_intersections(np.array(Planefrom4points_us), np.array(Planefrom4points), sun)
                                rotated_Coordinate4points = rotate_follow(intersections, theta)
                                cos = get_cos_theta(np.array(Planefrom4points))
                                targetbd = Polygon(rotated_Coordinate4points)
                                s_facade[u] = s_facade[u].union(targetbd) 
                            s_facade[u] = polygon_facade.intersection(s_facade[u]) 
                            s_facade_value[u] = s_facade[u].area 
                        
                        Planefrom4points = [building_target_dingmian[0],building_target_dingmian[1],building_target_dingmian[2],building_target_dingmian[3]]
                        polygon_roof = Polygon(building_target_dingmian)
                        H_value_to_assign = building_target_dingmian[0][2]
                        H_value_near_building = building_near_dingmian[0][2]
                        if H_value_to_assign > H_value_near_building:
                            continue  
                        for g in range(0,len(z_box),1):
                            vertex = z_box[g]
                            Planefrom4points_us = [list(building_near[vertex]),list(building_near[vertex+1]),
                                                   list(building_near_dingmian[vertex+1]),list(building_near_dingmian[vertex])]
                            Planefrom4points_us[0][2] = H_value_to_assign
                            Planefrom4points_us[1][2] = H_value_to_assign
                            if is_behind(Planefrom4points_us, Planefrom4points, sun):
                                continue        
                            intersections = calculate_intersections(np.array(Planefrom4points_us), np.array(Planefrom4points), sun)
                            try:
                                if np.any(np.isnan(intersections)):
                                    print(f"Skipping building ID: {unique_building_id}")
                                    continue  # Skip this iteration if NaN values present
                                targetbd = Polygon(intersections)
                            except:
                                continue
                            try:
                                s_roof = s_roof.union(targetbd)
                                continue
                            except Exception as e:
                                print(f"An error occurred: {e}")
                                print(f"Skipping building ID: {unique_building_id}")
                                continue
                            s_roof = s_roof.union(targetbd) 
                        try:
                            s_roof = polygon_roof.intersection(s_roof)
                            s_roof_value = s_roof.area
                            s_roof_str = wkt_dumps(s_roof)
                            continue
                        except Exception as e:
                            print(f"Error in final intersection or area calculation: {e}")
                            print(unique_building_id)
                            continue
                    for e in range(0, len(u_box), 1):
                        u = u_box[e]

                    cell_key = f'cell_{r}_{w}'
                    building_key = f'{unique_building_id}_building_{round(original_roof_area)}'
                    day_key = f'day_{k}'
                    hour_key = f'hour_{i}'
                    if cell_key not in results:
                        results[cell_key] = {}
                    if building_key not in results[cell_key]:
                        results[cell_key][building_key] = {}
                    if day_key not in results[cell_key][building_key]:
                        results[cell_key][building_key][day_key] = {}
                    if hour_key not in results[cell_key][building_key][day_key]:
                        results[cell_key][building_key][day_key][hour_key] = {
                            f'{unique_building_id}_roof_{round(original_roof_area)}': original_roof_area,
                            'facade': {},
                        }
                    for e in range(len(u_box)):
                        u = u_box[e]
                        shading_area = s_facade_value[u]
                        remaining_area = original_facade_areas[u] - shading_area
                        results[cell_key][building_key][day_key][hour_key]['facade'][f'{u}_{round(original_facade_areas[u])}'] = s_facade_value[u]

                    shading_area_roof = s_roof_value
                    remaining_roof_area = original_roof_area - shading_area
                    results[cell_key][building_key][day_key][hour_key][f'{unique_building_id}_roof_{round(original_roof_area)}'] = s_roof_value

                    buildings_analyzed += 1
            unique_building_id +=1
end_time = time.time()
elapsed_time = end_time - start_time



building_loc_metadata_converted = convert_to_list(building_loc_metadata)
with open(f'results/{test_case}_geometry.json', 'w') as json_file:
    json.dump(results, json_file, indent=4)
with open(f'results/{test_case}_results_{elapsed_time:.3f}.json', 'w') as json_file:
    json.dump(building_loc_metadata_converted, json_file, indent=4)